## Desarrollo de algoritmo para simulación.
<br>
<p>El desarrollo de este algoritmo fue llevado a cabo durante el semestre agosto-diciembre 2020, como parte de la asignatura de robotica.</p>
<br>
<p>
    <li>Alumno : Jose Alfredo de Jesus Aguiar Arce.</li>
    <li>Numero de control : 15400806.</li>
    <li>Materia : Robotica.</li>
    <li>Maestro : Dr. Antonio Guzman Navarrete.</li>
</p>

<br>Requiere que los archivos 'sim.py', 'simConst.py', 'remoteapi.dll' estén alojados en la misma carpeta que este cuaderno de Jupyter.
<br>Desde CoppeliaSim, abrir la escena 'DeskBot_scena.ttt'

### Instrucciones para preparar la simulacion.
<br>1. Abra la escena 'DeskBot_scena.ttt'
<br>2. Seleccionando la base del robot desde la jerarquía de escena, presione botón derecho del mouse y agregue un archivo de script mediante <b> Add -> Associated child script -> Non threaded </b>. Aparecerá un pequeño ícono de documento junto al nombre del robot en la escena de jerarquía.
<br>3. En el script es posible incluir código de programación, escritos en lenguaje LUA. Para nuestro caso, todo el código que requeriremos es habilitar el API remoto, asignando un puerto de comunicación. En la función <b>sysCall_init()</b> agregue la siguiente línea:
<br> 
<font face = "Monospace"> <pre> simRemoteApi.start(19999)</font>
<br> Proceda a continuación con las actividades:

In [1]:
# importamos las librerías necesarias
import sim          # librería para conectar con CoppeliaSim
import sympy as sp  # librería para cálculo simbólico
import numpy as np

<p>Configuraciones del servidor local</p>

In [2]:
#Variables para configurar entorno del servidor y port
localhost = '127.0.0.1'  #Servidor local de la computadora
port = 19999  #Puerto para coneccion con V-rep

### Configuramos el servidor y el port

El siguiente codigo deberia ejecutarse una vez que se presiona play en la simulacion, de funcionar correctamente el cliente quedara conectado con la API de V-rep.

In [3]:
def connect(port):
# Establece la conexión a VREP
# port debe coincidir con el puerto de conexión en VREP
# retorna el número de cliente o -1 si no puede establecer conexión
    sim.simxFinish(-1) # Cierra ports abiertos en caso de haber
    clientID=sim.simxStart(localhost,port,True,True,2000,5) # Conectarse
    if clientID == 0: print("conectado a", port)
    else: print("no se pudo conectar")
    return clientID


In [143]:
# Requerimos los manejadores para las articulaciones y el Dummy
clientID = connect(19999)

conectado a 19999


### Configuramos el programa y obtenemos objetos de la simulacion.

obtendermos las juntas y objetos de la simulacion para asignarlos a variables con las que nos podremos referir en el codigo.
De esta forma trabajaremos con POO (Programacion Orientada a Objetos) en nuestro programa.

In [109]:
#Variables para configurar entorno 
#Estas variables se agregaron dado que la simulacion fuera a modificarse y por ende sus valores 
#tales como rangos de juntas prismaticas, nombres de juntas etc.

#Como esta nombrada la junta revoluta de dicho eslabon
nombre_junta_eslabon_1 = 'eslabon_1' #la primera junta revoluta acoplada a la corredera
nombre_junta_eslabon_2 = 'eslabon_2' #la segunda junta revoluta acoplada a el eslabon anterior

#Como esta nombrada la junta prismatica de dicho eslabon
nombre_junta_corredera = 'Corredera'
nombre_junta_efector = 'efector_final'

#Valores de los rangos minimos y maximos de juntas prismaticas (en metros) dado que las funciones se encargan
#de transformar en decimales necesarios 
min_corredera,max_corredera = [-20,20]
min_efector,max_efector = [-4,4]

print(f'El nombre en la simulacion de la junta revoluta del eslabon 1 debe ser {nombre_junta_eslabon_1}')
print(f'El nombre en la simulacion de la junta revoluta del eslabon 2 debe ser {nombre_junta_eslabon_2}')
print(f'El nombre en la simulacion de la junta prismatica de la corredera debe ser {nombre_junta_corredera}, limite minimo : {min_corredera} mts, limite maximo {max_corredera}')
print(f'El nombre en la simulacion de la junta prismatica del efector final debe ser {nombre_junta_efector}, limite minimo : {min_efector} mts, limite maximo {max_efector}')

El nombre en la simulacion de la junta revoluta del eslabon 1 debe ser eslabon_1
El nombre en la simulacion de la junta revoluta del eslabon 2 debe ser eslabon_2
El nombre en la simulacion de la junta prismatica de la corredera debe ser Corredera, limite minimo : -20 mts, limite maximo 20
El nombre en la simulacion de la junta prismatica del efector final debe ser efector_final, limite minimo : -4 mts, limite maximo 4


In [110]:
#Manejadores de juntas revolutas

# Obtenemos el manejador de las juntas (joints), y se asignaran a una variable
returnCode,junta_1 = sim.simxGetObjectHandle(clientID,nombre_junta_eslabon_1,sim.simx_opmode_blocking)
returnCode,junta_2 = sim.simxGetObjectHandle(clientID,nombre_junta_eslabon_2,sim.simx_opmode_blocking)
#Imprimimos los id de cada junta
print(f"EL ID de junta_1 = {junta_1}, el ID de junta_2 = {junta_2}")


#Manejadores de juntas prismaticas

#Obtenemos el manejador de la corredera que se coloca en la base
returnCode,corredera = sim.simxGetObjectHandle(clientID,nombre_junta_corredera,sim.simx_opmode_blocking)
#Obtenemos el manejador del efector final
returnCode,efector_cilindro = sim.simxGetObjectHandle(clientID,nombre_junta_efector,sim.simx_opmode_blocking)
#Imprimimos el id de cada junta
print(f"El ID de corredera es = {corredera}, el ID del efector final es {efector_cilindro}")

EL ID de junta_1 = 17, el ID de junta_2 = 18
El ID de corredera es = 16, el ID del efector final es 19


### Posicionamiento en el simulador

Funciones y demostraciones sobre el como se realiza la simulacion sin entrar en detalles de cinematica aun.
Son las funciones necesarias para modificar en la simulacion posiciones.

In [80]:
#la posicion para juntas revolutas se asignara mediante grados 
#Los angulos deben enviarse en radianes, dado que se trabaja con grados se usara la expresion de la manera siguiente
# angulo_en_radianes = angulo_en_grados * pi / 180

angulo_1 = 0 * np.pi/180  #El angulo convertido a radianes para la junta 1.
angulo_2 = 0* np.pi/180 #El angulo convertido a radianes para la junta 2.

retCode = sim.simxSetJointTargetPosition(clientID, junta_1, angulo_1, sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, junta_2, angulo_2, sim.simx_opmode_oneshot)

#La posicion para juntas prismaticas se asignara representada en distancia.
#La distancia debe ser representada entre [pos.min] y [pos.range] , ambos parametros que se han definido en la simulacion, 
#asignados al elemento (joint) corredera.

#Dado que se debe encontrar expresado en Metros la ecuacion sera la siguiente
#distancia_en_metros = distancia / 100
#Para esta configuracion distancia debe ir de 'min_corredera' hasta 'max_corredera' en el caso de la corredera
distancia = 0 / 100 
#Para esta configuracion distancia debe ir de 'min_efector' hasta 'max_efector' en el caso del efector final
distancia_efector = -4 / 100 

retCode = sim.simxSetJointTargetPosition(clientID, corredera,distancia, sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, efector_cilindro,distancia_efector, sim.simx_opmode_oneshot)
print(retCode)

0


Dadas las observaciones anteriores se escribieron dos metodos para ahorrar escritura y simplificar la implementacion

In [145]:
def mover_junta_revoluta(junta,angulo):
    """
    Esta funcion permite realizar una rotacion en la junta revoluta desde el angulo actual hasta el deseado (grados)
    """
    radianes = angulo * np.pi/180
    
    #Regresa 0 si se ejecuta correctamente
    return sim.simxSetJointTargetPosition(clientID, junta, radianes, sim.simx_opmode_oneshot) 

#Ejemplo de como se usa este metodo
mover_junta_revoluta(junta_1,-90)
mover_junta_revoluta(junta_2,90)

def mover_junta_prismatica(junta,distancia_desde_origen):
    """
    Esta funcion permite realizar un dezplazamiento en la junta prismatica (debe estar expresada en centimetros)
    en un rango entre [Pos Min - Pos Range] de la junta prismatica en cuestion
    """
    
    distancia = distancia_desde_origen / 100
    #Regresa 0 si se ejecuta correctamente
    return sim.simxSetJointTargetPosition(clientID, junta,distancia, sim.simx_opmode_oneshot)

#Ejemplo de como se usa este metodo
mover_junta_prismatica(corredera,20)
mover_junta_prismatica(efector_cilindro,4)

0

Algunas utilidades realizadas para facilitar la escritura de posiciones comunes

In [148]:
def abajo_corredera():
    """
    Envia la corredera a su posicion minima, en el eje Z.
    """
    mover_junta_prismatica(corredera,min_corredera)


def arriba_corredera():
    """
    Envia la corredera a su posicion maxima, en el eje Z.
    """
    mover_junta_prismatica(corredera,min_corredera)

def abrir_efector():
    """
    activa el efector , es decir lo baja.
    """
    mover_junta_prismatica(efector_cilindro,min_efector)

def cerrar_efector():
    """
    desactiva el efector , es decir lo sube. 
    """
    mover_junta_prismatica(efector_cilindro,max_efector)


def reiniciar_robot():
    """
    Deja el robot en su posicion inicial 
    """
    cerrar_efector()
    abajo_corredera()
    mover_junta_revoluta(junta_1,0)
    mover_junta_revoluta(junta_2,0)


### Cinematica directa del modelo por D-H

Se realiza un algoritmo de Denavit-Hartenberg para resolver el modelo cinematico directo del robot.